In [1]:
import os
import re
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
# load data
text = open(os.path.join(os.getcwd(), 'data/shakespeare.txt')).read()

# remove sonnet numbers and convert to lowercase
text = re.sub(r'[0-9]+', '', text) 
text = text.lower()

# create mapping of unique chars to integers
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

n_chars = len(text) # total length of dataset
n_vocab = len(chars) # number of unique characters

In [3]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 40
dataX = []
dataY = []

# use sliding window approach
for i in range(0, n_chars - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    
n_sequences = len(dataX)
print(dataY)

[16, 30, 1, 34, 16, 1, 15, 16, 30, 20, 29, 16, 1, 20, 25, 14, 29, 16, 12, 30, 16, 6, 0, 31, 19, 12, 31, 1, 31, 19, 16, 29, 16, 13, 36, 1, 13, 16, 12, 32, 31, 36, 3, 30, 1, 29, 26, 30, 16, 1, 24, 20, 18, 19, 31, 1, 25, 16, 33, 16, 29, 1, 15, 20, 16, 6, 0, 13, 32, 31, 1, 12, 30, 1, 31, 19, 16, 1, 29, 20, 27, 16, 29, 1, 30, 19, 26, 32, 23, 15, 1, 13, 36, 1, 31, 20, 24, 16, 1, 15, 16, 14, 16, 12, 30, 16, 6, 0, 19, 20, 30, 1, 31, 16, 25, 15, 16, 29, 1, 19, 16, 20, 29, 1, 24, 20, 18, 19, 31, 1, 13, 16, 12, 29, 1, 19, 20, 30, 1, 24, 16, 24, 26, 29, 36, 9, 0, 13, 32, 31, 1, 31, 19, 26, 32, 1, 14, 26, 25, 31, 29, 12, 14, 31, 16, 15, 1, 31, 26, 1, 31, 19, 20, 25, 16, 1, 26, 34, 25, 1, 13, 29, 20, 18, 19, 31, 1, 16, 36, 16, 30, 6, 0, 17, 16, 16, 15, 3, 30, 31, 1, 31, 19, 36, 1, 23, 20, 18, 19, 31, 3, 30, 1, 17, 23, 12, 24, 16, 1, 34, 20, 31, 19, 1, 30, 16, 23, 17, 7, 30, 32, 13, 30, 31, 12, 25, 31, 20, 12, 23, 1, 17, 32, 16, 23, 6, 0, 24, 12, 22, 20, 25, 18, 1, 12, 1, 17, 12, 24, 20, 25, 16, 1, 3

In [4]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_sequences, seq_length, 1))

# normalize data to range (0, 1)
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)
print(X)
print(y)

[[[0.02631579]
  [0.02631579]
  [0.02631579]
  ...
  [0.81578947]
  [0.84210526]
  [0.76315789]]

 [[0.02631579]
  [0.02631579]
  [0.02631579]
  ...
  [0.84210526]
  [0.76315789]
  [0.42105263]]

 [[0.02631579]
  [0.02631579]
  [0.02631579]
  ...
  [0.76315789]
  [0.42105263]
  [0.78947368]]

 ...

 [[0.42105263]
  [0.07894737]
  [0.78947368]
  ...
  [0.02631579]
  [0.60526316]
  [0.68421053]]

 [[0.07894737]
  [0.78947368]
  [0.02631579]
  ...
  [0.60526316]
  [0.68421053]
  [0.86842105]]

 [[0.78947368]
  [0.02631579]
  [0.44736842]
  ...
  [0.68421053]
  [0.86842105]
  [0.42105263]]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
# define the LSTM model 
# (single layer with 150 units, followed by dense output layer)
model = Sequential()
model.add(LSTM(150, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [6]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [7]:
# train the model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
97635/97635 [==============================] - 40s 411us/step - loss: 2.9365

Epoch 00001: loss improved from inf to 2.93653, saving model to weights-improvement-01-2.9365.hdf5
Epoch 2/20
97635/97635 [==============================] - 39s 396us/step - loss: 2.7409

Epoch 00002: loss improved from 2.93653 to 2.74086, saving model to weights-improvement-02-2.7409.hdf5
Epoch 3/20
97635/97635 [==============================] - 39s 395us/step - loss: 2.6653

Epoch 00003: loss improved from 2.74086 to 2.66525, saving model to weights-improvement-03-2.6653.hdf5
Epoch 4/20
97635/97635 [==============================] - 39s 395us/step - loss: 2.6109

Epoch 00004: loss improved from 2.66525 to 2.61087, saving model to weights-improvement-04-2.6109.hdf5
Epoch 5/20
97635/97635 [==============================] - 39s 397us/step - loss: 2.5745

Epoch 00005: loss improved from 2.61087 to 2.57450, saving model to weights-improvement-05-2.5745.hdf5
Epoch 6/20
97635/97635 [====================